In [36]:
import pandas as pd
import re
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline 
#generate side graph

In [37]:
artists = pd.read_csv(open("artists.csv"))
artworks = pd.read_csv(open("artworks.csv"))

# MoMA Acquisitions

The goal of this file is to answer one big question: **how should MoMA improve its future art acquisition?**

First, choose selective data into a new dataframe and classify donors into different types.
Overview the data number of rows and columns as well as data types

In [106]:
artwork = artworks[["Artwork ID", "Date","Acquisition Date","Credit","Department"]]
print(artwork.shape)
print(artwork.dtypes)

(130262, 5)
Artwork ID           int64
Date                object
Acquisition Date    object
Credit              object
Department          object
dtype: object


Second, classify donors into six categories with regex expressions

In [108]:
def donor_type(donor):
    if re.search(r'Gift',str(donor)) or re.search(r', gift',str(donor)) :
        return "Gift"
    elif re.search(r'^Purchase+$',str(donor)):
        return "Purchase"
    elif re.search(r'Purchase and partial gift',str(donor)) or re.search(r'partial gift',str(donor)):
        return "Purchase and partial gift"
    elif re.search(r'National Endowment',str(donor)):
        return "National Endowment"
    elif re.search(r'Given anonymously',str(donor)):
        return "Given anonymously"
    else:
        return "Other"

artwork['Donor Type'] = artwork['Credit'].apply(donor_type)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


## Sub-question 1: What is donors' acquisition trend by year?

In [120]:
donor_counts = artwork.groupby("Donor Type").size()
print(donor_counts.iloc[:10])
print(artwork["Acquisition Date"])

Donor Type
Gift                         66406
Given anonymously             1581
National Endowment             295
Other                        53490
Purchase                      8404
Purchase and partial gift       86
dtype: int64
0         1996-04-09
1         1995-01-17
2         1997-01-15
3         1995-01-17
4         1997-01-15
             ...    
130257    2008-10-08
130258    2008-10-08
130259    2008-10-08
130260    2008-10-08
130261    2008-10-08
Name: Acquisition Date, Length: 130262, dtype: object
